In [13]:
from core import vector_store, embed_model, llm

from llama_index.core import VectorStoreIndex

index = VectorStoreIndex.from_vector_store(vector_store, embed_model=embed_model)
query_engine = index.as_query_engine(llm=llm, similarity_top_k=2)
retriever = index.as_retriever(similarity_top_k=10)

In [3]:
response = retriever.retrieve('сколько богатырей выходят из моря?')

response

[NodeWithScore(node=TextNode(id_='1f84e83d-9910-471c-90a1-a5a00c476ef4', embedding=None, metadata={'file_path': '/home/ubuntu/ascheglov/hackathon-ml-search/elastic/../data/texts/web-server.md', 'file_name': 'web-server.md', 'file_type': 'text/markdown', 'file_size': 22475, 'creation_date': '2025-01-09', 'last_modified_date': '2025-01-09', 'document_title': ' "Comprehensive Guide to Web Servers: Diving into HTTP, Headers, Data Compression, and Beyond"', 'prev_section_summary': "1. SSH Key Generation, Management, and Transfer: This guide is about securing cross-computer access using SSH keys.\n\n   2. Granting Access to Multiple Users: To grant access to multiple users under one account, simply add their public key on a new line in the 'authorized_keys' file.\n\n   3. Testing Access: After adding the public key, exit the server and attempt to re-enter - this time, you won't be asked for a password as the server already has your key.\n\n   4. Denying Root User Access via Password: To prev

In [19]:
from llama_index.core.prompts.base import PromptTemplate
from llama_index.core.prompts.prompt_type import PromptType

qa_prompt_tmpl = PromptTemplate(
    (
        "Context information is below.\n"
        "---------------------\n"
        "{context_str}\n"
        "---------------------\n"
        "Given the context information and not prior knowledge, "
        "answer the query.\n"
        "Use russian for answer.\n"
        "Query: {query_str}\n"
        "Answer: "
    ),
    prompt_type=PromptType.QUESTION_ANSWER
)

refine_template = PromptTemplate(
    (
        "The original query is as follows: {query_str}\n"
        "We have provided an existing answer: {existing_answer}\n"
        "We have the opportunity to refine the existing answer "
        "(only if needed) with some more context below.\n"
        "------------\n"
        "{context_msg}\n"
        "------------\n"
        "Given the new context, refine the original answer to better "
        "answer the query. "
        "If the context isn't useful, return the original answer.\n"
        "Use russian for answer.\n"
        "Refined Answer: "
    ),
    prompt_type=PromptType.REFINE
)

query_engine.update_prompts(
    {
        "response_synthesizer:text_qa_template": qa_prompt_tmpl,
        "response_synthesizer:refine_template": refine_template,
    },
)

In [21]:
answer = query_engine.query('сколько богатырей выходят из моря?')

answer

Response(response=' Из сведений, которые предоставлены, невозможно определить, сколько богатырей выходят из моря.', source_nodes=[NodeWithScore(node=TextNode(id_='1f84e83d-9910-471c-90a1-a5a00c476ef4', embedding=None, metadata={'file_path': '/home/ubuntu/ascheglov/hackathon-ml-search/elastic/../data/texts/web-server.md', 'file_name': 'web-server.md', 'file_type': 'text/markdown', 'file_size': 22475, 'creation_date': '2025-01-09', 'last_modified_date': '2025-01-09', 'document_title': ' "Comprehensive Guide to Web Servers: Diving into HTTP, Headers, Data Compression, and Beyond"', 'prev_section_summary': "1. SSH Key Generation, Management, and Transfer: This guide is about securing cross-computer access using SSH keys.\n\n   2. Granting Access to Multiple Users: To grant access to multiple users under one account, simply add their public key on a new line in the 'authorized_keys' file.\n\n   3. Testing Access: After adding the public key, exit the server and attempt to re-enter - this ti

In [20]:
from llama_index.core.prompts import display_prompt_dict

prompts_dict = query_engine.get_prompts()
display_prompt_dict(prompts_dict)

**Prompt Key**: response_synthesizer:text_qa_template<br>**Text:** <br>

Context information is below.
---------------------
{context_str}
---------------------
Given the context information and not prior knowledge, answer the query.
Use russian for answer.
Query: {query_str}
Answer: 


<br><br>

**Prompt Key**: response_synthesizer:refine_template<br>**Text:** <br>

The original query is as follows: {query_str}
We have provided an existing answer: {existing_answer}
We have the opportunity to refine the existing answer (only if needed) with some more context below.
------------
{context_msg}
------------
Given the new context, refine the original answer to better answer the query. If the context isn't useful, return the original answer.
Use russian for answer.
Refined Answer: 


<br><br>